In [1]:
from matplotlib.patches import Patch
import sys, time, math
import matplotlib.pyplot as plt
import numpy as np
sys.path.append('/home/groups/ZuckermanLab/jalim/instalLocal/celltraj/celltraj')
import jcTrajectory_CP as cellTraj
import nucSCTM_CP as nucMsksCCR
from csaps import csaps
import umap, string, pickle 
from datetime import date
from skimage import io, measure
from IPython.display import clear_output
from matplotlib.animation import FuncAnimation, FFMpegWriter
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from sklearn.decomposition import PCA

2024-05-17 11:35:46.631102: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-17 11:35:46.740700: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-17 11:36:00.706960: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
trajl = 1 # Trajectory length for the morphodynamical embedding 
today = date.today()
date2day = today.strftime("%b%d-%Y")
sysName = 'LI204601_P'
figid = sysName+'_tlen'+str(trajl)+'_'+date2day
conditions = ['A2'] # LIGANDS (CONDITIONS)
tmSet = ['EGF1']
nConditions = len(tmSet) # Total number of Ligand Conditions
inds_tmSet = [i for i in range(nConditions)]
inds_tmSet = np.array(inds_tmSet).astype(int)
nfovs = 1
fovs = [i for i in range(1, nfovs + 1)]
fovs = np.array(fovs).astype(int)
dateSet = ['']
pathSet = ['/home/groups/ZuckermanLab/jalim/LI204601_INCUCYTE/segsCellPose/bayesianTrackTest/']
imagingSet = [0 for i in range(nConditions)]
modelList = [None]*(nfovs*(nConditions))
modelList_conditions = np.zeros(nfovs*(nConditions)).astype(int)

In [3]:
i = 0
icond = 0
for cond in conditions:
    for fov in fovs:
        modelList_conditions[i] = icond
        modelList[i] = pathSet[imagingSet[icond]]+sysName+'_'+cond+'_'+str(fov)+dateSet[imagingSet[icond]]
        #print("Models: ",modelList[i])
        i = i + 1
    icond = icond + 1

In [4]:
nmodels = len(modelList)
modelSet = [None]*nmodels
indgood_models = np.array([]).astype(int)

for i in range(nmodels):
    try:
        objFile = modelList[i]+'.obj'
        objFileHandler = open(objFile,'rb')
        modelSet[i] = pickle.load(objFileHandler)
        print('loaded '+objFile+' with '+str(modelSet[i].cells_indSet.size)+' cells')
        objFileHandler.close()
        test = len(modelSet[i].linSet)
        indgood_models = np.append(indgood_models,i)
    except:
        print("ERROR in reading *.obj files")
        sys.exit(0)

loaded /home/groups/ZuckermanLab/jalim/LI204601_INCUCYTE/segsCellPose/bayesianTrackTest/LI204601_P_A2_1.obj with 51054 cells


In [5]:
# Total number of frames (image snapshots) in one condition per FOVs
nframes = 193 
cellnumber_stdSet = np.ones(nmodels)*np.inf
# range of frame indices where cell numbers are higher: ~70-98%
sframe = 70.*nframes/100.; sframe = math.ceil(sframe)
eframe = 98.5*nframes/100.; eframe = math.ceil(eframe)
cellnumber_frames = np.arange(sframe, eframe).astype(int)
cellnumber_std_cut = .50 # This was set to 0.10 by Jeremy 
frames = np.arange(nframes)
# Abscissas at which smoothing will be done using CSAPS package
abSmooth = np.linspace(frames[0], frames[-1], 10000)

for i in indgood_models:
    ncells = np.zeros(nframes)
    ncells_smooth = np.zeros_like(ncells)
    for iS in range(nframes):
        ncells[iS]=np.sum(modelSet[i].cells_frameSet == iS)
    # Cubic Spline Approximation (CSAPS) to smoothen the data
    splfov = csaps(frames, ncells/ncells[0], abSmooth, smooth = 0.98) # Scaled by ncells[0] to avoid large numbers
    ncells_smooth = splfov*ncells[0] # smoothened cell numbers reverse scaled back to original
    cellnumber_std = np.std(ncells[cellnumber_frames] - ncells_smooth[cellnumber_frames])/np.mean(ncells[cellnumber_frames])
    cellnumber_stdSet[i] = cellnumber_std # Standard Deviation in Cell Numbers

In [6]:
indhigh_std = np.where(cellnumber_stdSet > cellnumber_std_cut)[0]
indgood_models = np.setdiff1d(indgood_models, indhigh_std)
#indgood_models = np.setdiff1d(indgood_models, np.array([51]).astype(int)) #missing comdx for EGF0_4

# get cell counts
nf = len(tmSet)
inds_tmSet_models = np.zeros(nmodels).astype(int)
inds_imagingSet_models = np.zeros(nmodels).astype(int)
i = 0
icond = 0
for cond in conditions:
    for fov in fovs:
        inds_tmSet_models[i] = inds_tmSet[icond]
        inds_imagingSet_models[i] = imagingSet[icond]
        i = i + 1
    icond = icond + 1

In [7]:
for i in indgood_models:
    if inds_imagingSet_models[i] == 0:
        modelSet[i].Xf[np.isnan(modelSet[i].Xf)] = 0.0 #just replace with zeros for now? Not sure best...

nfeat_com = 3
Xf_com0 = np.zeros((0, nfeat_com))
for i in indgood_models:
    if inds_imagingSet_models[i] == 0:
        Xf_com0 = np.append(Xf_com0,modelSet[i].Xf_com, axis = 0)

av_dx = np.nanmean(Xf_com0[:, 0])
std_dx = np.nanstd(Xf_com0[:, 0])
for i in indgood_models:
    modelSet[i].Xf_com[:, 0] = (modelSet[i].Xf_com[:, 0] - av_dx)/std_dx

In [8]:
wctm = cellTraj.Trajectory() # import Trajectory object 

nfeat = modelSet[indgood_models[0]].Xf.shape[1]
Xf = np.zeros((0, nfeat))
indtreatment = np.array([])
indcellSet = np.array([])
for i in indgood_models:
    if inds_imagingSet_models[i] == 0:
        Xf = np.append(Xf, modelSet[i].Xf, axis = 0)
        indtreatment = np.append(indtreatment, i*np.ones(modelSet[i].Xf.shape[0]))
        indcellSet = np.append(indcellSet, modelSet[i].cells_indSet)

indtreatment = indtreatment.astype(int)
indcellSet = indcellSet.astype(int)

In [9]:
varCutOff = 10 # High Variance (of Covariance Matrix) Number of Components  
pca = PCA(n_components = varCutOff) 
pca.fit(Xf) # builds the covariance matrix and "fits" the principal components
Xpca = pca.transform(Xf) # transforms the data into the PCA representation
nPCs = Xpca.shape[1]

In [10]:
wctm.Xpca = Xpca
wctm.pca = pca
for i in indgood_models:
    if inds_imagingSet_models[i] == 0:
        indsf = np.where(indtreatment == i)[0]
        modelSet[i].Xpca = Xpca[indsf, :]

indgood_models = indgood_models[np.where(inds_imagingSet_models[indgood_models] == 0)[0]]

In [11]:
self = wctm
#wctm.trajl = trajl
all_trajSet = [None]*nmodels
for i in indgood_models:
    #modelSet[i].get_unique_trajectories(verbose = True)
    modelSet[i].get_unique_trajectories()

tracked cell 50891, 25 tracks, 46600 left
tracked cell 50763, 23 tracks, 42400 left
tracked cell 50735, 51 tracks, 41500 left
tracked cell 50076, 1 tracks, 37700 left
tracked cell 49741, 1 tracks, 37600 left
tracked cell 49307, 1 tracks, 37500 left
tracked cell 49056, 18 tracks, 37400 left
tracked cell 48763, 1 tracks, 37100 left
tracked cell 48214, 1 tracks, 36900 left
tracked cell 48008, 1 tracks, 36800 left
tracked cell 47562, 1 tracks, 36500 left
tracked cell 46286, 1 tracks, 35900 left
tracked cell 46119, 3 tracks, 35700 left
tracked cell 45624, 1 tracks, 35600 left
tracked cell 42924, 7 tracks, 34200 left
tracked cell 41889, 2 tracks, 33800 left
tracked cell 41728, 1 tracks, 33700 left
tracked cell 40948, 1 tracks, 33300 left
tracked cell 39770, 1 tracks, 32900 left
tracked cell 39018, 1 tracks, 32500 left
tracked cell 38257, 1 tracks, 32300 left
tracked cell 38063, 6 tracks, 32200 left
tracked cell 37504, 1 tracks, 32000 left
tracked cell 35855, 2 tracks, 31300 left
tracked cell

In [44]:
for i in indgood_models:
    # Set of all single-cell trajectories with absolute (irrespective of frames) cell indices
    num_single_cell_trajs = len(modelSet[i].trajectories)
    single_cell_traj_lengths = []
    for itraj in range(num_single_cell_trajs):
        single_cell_traj_inds = modelSet[i].trajectories[itraj] # Indices of single-cell trajectory across frames
        single_cell_traj_length = len(single_cell_traj_inds) # Length of a single-cell trajectory
        single_cell_traj_lengths.append(single_cell_traj_length)  
    indtrajs = np.argsort(single_cell_traj_lengths) # Length ordered array of unique single-cell trajectories 

In [13]:
############ Find the longest single-cell trajectory ############
longest_trajectory = max(single_cell_traj_lengths) 
if longest_trajectory > nframes: 
    print("Error: longest single-cell trajectory can't exceed total number of frames")
    sys.exit(0)
#print("Longest single-cell trajectory length: ",longest_trajectory)
ind_longest_trajectory = np.argmax(single_cell_traj_lengths) # Index of the longest single-cell trajectory
#print("Index of longest single-cell trajectory: ",ind_longest_trajectory)
cell_inds_longest_traj = modelSet[i].trajectories[ind_longest_trajectory] # Cell indices
#print("Cell indices of the longest single-cell trajectory: ",cell_inds_longest_traj)

In [14]:
# Import single-cell trajectory model from cell-cycle reporter' class
sctm = nucMsksCCR.nucCellMasksCCRimgs()
sctm.visual = False
start_frame = 0
sysName = 'LI204601_G'
filepath = '/home/groups/ZuckermanLab/jalim/LI204601_INCUCYTE/ratioNucCyto/'

In [15]:
model = filepath+sysName+'_'+conditions[0]+'_1.h5'
print("Model name:", model)
# Load cell cycle reporter and nuclear images and cytoplasm masks
print('initializing...')
sctm.initialize(model, sysName)
# Get all the frames (193 at intervals of 15 minutes) for each model (condition)
sctm.get_frames()
end_frame = sctm.maxFrame
# Get the cell-cycle reporter image, cytoplasmic & nuclear masks
sctm.get_imageSet(start_frame, end_frame)

Model name: /home/groups/ZuckermanLab/jalim/LI204601_INCUCYTE/ratioNucCyto/LI204601_G_A2_1.h5
initializing...
Frame 0 has 1 images...
Frame 1 has 1 images...
Frame 2 has 1 images...
Frame 3 has 1 images...
Frame 4 has 1 images...
Frame 5 has 1 images...
Frame 6 has 1 images...
Frame 7 has 1 images...
Frame 8 has 1 images...
Frame 9 has 1 images...
Frame 10 has 1 images...
Frame 11 has 1 images...
Frame 12 has 1 images...
Frame 13 has 1 images...
Frame 14 has 1 images...
Frame 15 has 1 images...
Frame 16 has 1 images...
Frame 17 has 1 images...
Frame 18 has 1 images...
Frame 19 has 1 images...
Frame 20 has 1 images...
Frame 21 has 1 images...
Frame 22 has 1 images...
Frame 23 has 1 images...
Frame 24 has 1 images...
Frame 25 has 1 images...
Frame 26 has 1 images...
Frame 27 has 1 images...
Frame 28 has 1 images...
Frame 29 has 1 images...
Frame 30 has 1 images...
Frame 31 has 1 images...
Frame 32 has 1 images...
Frame 33 has 1 images...
Frame 34 has 1 images...
Frame 35 has 1 images...


In [16]:
def cross_correlation(cropped_reporter_image, cropped_nuclear_image):

    # Flatten the images if they are 2D
    if cropped_reporter_image.ndim == 2:
        cropped_reporter_image = cropped_reporter_image.flatten()
    if cropped_nuclear_image.ndim == 2:
        cropped_nuclear_image = cropped_nuclear_image.flatten()
        
    # Compute correlation coefficient
    correlation_coefficient = np.corrcoef(cropped_reporter_image, cropped_nuclear_image)[0, 1]
    
    return correlation_coefficient

In [17]:
# Function to Z-normalize masked and non-masked images
def z_normalize(image):
    
    if np.ma.is_masked(image):
        # If the image is a masked array
        data = image.data
        mask = image.mask

        # Perform z-normalization on the data
        normalized_data = (data - np.nanmean(data)) / np.nanstd(data)

        # Create a new masked array with the normalized data and the original mask
        return np.ma.masked_array(normalized_data, mask = mask)
    else:
        # If the image is a regular array
        return (image - np.nanmean(image)) / np.nanstd(image)

In [18]:
def nuclei_within_cell_boundary(cropped_nuclear_mask, cropped_reporter_image,
                                cell_mask, hole_pixels = 3):
    # Apply closing to the nuclear mask 
    closed_nuclear_mask = closing(cropped_nuclear_mask, square(hole_pixels))

    # Label the regions in the closed nuclear mask
    labeled_nuclear_mask = label(closed_nuclear_mask)
    regions = regionprops(labeled_nuclear_mask, intensity_image = cropped_reporter_image)
    
    # Create a mask for only the nuclei within the cell boundary
    nuclei_within_cell_mask = np.zeros_like(cropped_nuclear_mask, dtype = bool)

    for region in regions:
        # Check if the labeled region centroid is within the cell mask
        if cell_mask[int(region.centroid[0]), int(region.centroid[1])]:
            # Combine masks using logical OR to keep the result as boolean
            nuclei_within_cell_mask = np.logical_or(nuclei_within_cell_mask, labeled_nuclear_mask == region.label)

    # Ensure the mask is a boolean type
    nuclei_within_cell_mask = nuclei_within_cell_mask.astype(bool)

    return nuclei_within_cell_mask

In [19]:
# Get frame indices where Microscopy imaging captures bad snaps 
model_indc = 0
n_cells_prev = 0
swap_imgs_frame_inds = []
for frame_indc in range(nframes): 
    # If the number of cells is equal in two consecutive frames
    n_cells_curr = np.sum(modelSet[model_indc].cells_frameSet == frame_indc)
    if n_cells_curr == n_cells_prev:
        swap_imgs_frame_inds.append(frame_indc)
    n_cells_prev = n_cells_curr

In [20]:
def get_single_cell_borders(self, indcells=None, bordersize=10):
        if not hasattr(self,'fmskSet'):
            print("Foreground masks not found: provide or derive them")
            sys.exit(0)
        nx = self.imgSet.shape[1]
        ny = self.imgSet.shape[2];
        if indcells is None:
            indcells = np.arange(self.cells_indSet.size).astype(int)
        ncells = indcells.size
        cellborder_imgs = [None]*ncells
        cellborder_msks = [None]*ncells
        cellborder_fmsks = [None]*ncells
        ii = 0
        ic = indcells
        sys.stdout.write('extracting cellborders from frame '+str(self.cells_frameSet[ic])+' image '+str(self.cells_imgfileSet[ic])+'\n')
        img = self.imgSet[self.cells_indimgSet[ic]]
        msk = self.mskSet[self.cells_indimgSet[ic]]
        fmsk = self.fmskSet[self.cells_indimgSet[ic]]
        cellblocks = self.get_cell_blocks(msk)

        xmin = np.max(np.array([cellblocks[self.cells_indSet[ic], 0, 0] - bordersize, 0]))
        xmax = np.min(np.array([cellblocks[self.cells_indSet[ic], 0, 1] + bordersize, nx - 1]))
        ymin = np.max(np.array([cellblocks[self.cells_indSet[ic], 1, 0] - bordersize, 0]))
        ymax = np.min(np.array([cellblocks[self.cells_indSet[ic], 1, 1] + bordersize, ny - 1]))

        imgcell = img[xmin:xmax, :]
        imgcell = imgcell[:, ymin:ymax]
        mskcell = msk[xmin:xmax, :]
        mskcell = mskcell[:, ymin:ymax]
        fmskcell = fmsk[xmin:xmax, :]
        fmskcell = fmskcell[:, ymin:ymax]
        tightmskcell = msk[cellblocks[self.cells_indSet[ic], 0, 0]:cellblocks[self.cells_indSet[ic], 0, 1], :]
        tightmskcell = tightmskcell[:, cellblocks[self.cells_indSet[ic], 1, 0]:cellblocks[self.cells_indSet[ic], 1, 1]]
        (values, counts) = np.unique(tightmskcell[np.where(tightmskcell > 0)], return_counts = True)
        icell = values[np.argmax(counts)].astype(int)
        mskcell = mskcell == icell

        cellborder_imgs[ii] = imgcell.copy()
        cellborder_msks[ii] = mskcell.copy()
        cellborder_fmsks[ii] = fmskcell.copy()
        self.cellborder_imgs = cellborder_imgs
        self.cellborder_msks = cellborder_msks
        self.cellborder_fmsks = cellborder_fmsks

        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax

In [21]:
def update(itt):
    swap_frames = False # Disable swapping by default 
    clear_output(wait = True)
    for row in axs:
        for ax in row:
            ax.clear()
    cs = axs[0][0].contourf(xx, yy, prob.T, cmap = plt.cm.Greys, extend = 'both')
    cs.cmap.set_over('black') 
    for ittt in range(itt-1):
        axs[0][0].arrow(xt[ittt, 0], xt[ittt, 1], xt[ittt+1, 0] - xt[ittt, 0], xt[ittt+1, 1] - xt[ittt, 1],
                 head_width = .2, linewidth = 1.0, color = plt.cm.winter(ittt/n_trajs), alpha = 1.0)
    indcells = np.array([cell_traj[itt]])
    print(f"Cell indices: {indcells[0]}")
    fid = modelSet[model_indc].cells_frameSet[indcells[0]] # Frame index 
    # Higher border margin for visualization only 
    get_single_cell_borders(modelSet[model_indc], indcells = indcells[0], bordersize = 50)
    
    imgcell = modelSet[model_indc].cellborder_imgs[0]
    mskcell = modelSet[model_indc].cellborder_msks[0]
    fmskcell = modelSet[model_indc].cellborder_fmsks[0]
    imgcell_znorm = z_normalize(imgcell)
    
    axs[0][1].imshow(np.ma.masked_where(fmskcell == 0, imgcell_znorm), cmap = plt.cm.seismic, clim = (-5, 5), alpha = 1.0)
    axs[0][1].imshow(np.ma.masked_where(fmskcell == 1, imgcell_znorm), cmap = plt.cm.gray, clim = (-5, 5), alpha = 0.6)

    # Get cell-cell (CC) and cell-surroundings (CS) borders
    ccborder, csborder = modelSet[model_indc].get_cc_cs_border(mskcell, fmskcell)
    nx = imgcell.shape[0]; ny = imgcell.shape[1]
    xxc, yyc = np.meshgrid(np.arange(nx), np.arange(ny), indexing='ij')
    cmskx = np.sum(np.multiply(xxc, mskcell))/np.sum(mskcell)
    cmsky = np.sum(np.multiply(yyc, mskcell))/np.sum(mskcell)
    axs[0][1].scatter(cmsky, cmskx, s = 600, color = 'black', marker = 'x', alpha = 0.6)

    get_single_cell_borders(modelSet[model_indc], indcells = indcells[0], bordersize = border_margin)
    
    imgcell = modelSet[model_indc].cellborder_imgs[0]
    mskcell = modelSet[model_indc].cellborder_msks[0]
    fmskcell = modelSet[model_indc].cellborder_fmsks[0]

    # Get cell-cell (CC) and cell-surroundings (CS) borders
    ccborder, csborder = modelSet[model_indc].get_cc_cs_border(mskcell, fmskcell)

    # Combine border information with cell interior to create a full mask for the entire cell
    cell_mask = np.logical_or(np.logical_or(ccborder, csborder), mskcell)
    
    reporter_image = sctm.imgSet[fid]
    nuclear_image = sctm.nucImgSet[fid]
    nuclear_mask = sctm.nmskSet[fid]
 
    swap_frames = fid in swap_imgs_frame_inds # If frame index if found in "swap_imgs_frame_inds"
    if swap_frames:
        # If bad imaging, swap Cell-cycle and Nuclear channel images and nuclear masks with the previous frame
        reporter_image = sctm.imgSet[fid-1]
        nuclear_image = sctm.nucImgSet[fid-1]
        nuclear_mask = sctm.nmskSet[fid-1] 
    else:
        reporter_image = sctm.imgSet[fid]
        nuclear_image = sctm.nucImgSet[fid]
        nuclear_mask = sctm.nmskSet[fid]
    
    reporter_image = np.array(reporter_image)
    nuclear_image = np.array(nuclear_image)

    x_min, x_max = modelSet[model_indc].xmin, modelSet[model_indc].xmax
    y_min, y_max = modelSet[model_indc].ymin, modelSet[model_indc].ymax

    # Crop reporter images, nuclear images, and masks according to single-cell positions
    cropped_reporter_image = reporter_image[x_min:x_max, y_min:y_max]
    cropped_nuclear_image = nuclear_image[x_min:x_max, y_min:y_max]
    cropped_nuclear_mask = nuclear_mask[x_min:x_max, y_min:y_max]

    # Mask out (ignore) the pixels that don't fit within the cell boundary
    cropped_reporter_image = np.ma.masked_where(cell_mask == 0, cropped_reporter_image)
    cropped_nuclear_image = np.ma.masked_where(cell_mask == 0, cropped_nuclear_image)
    
    # Z-normalize the area of interest within the images 
    cropped_reporter_image = z_normalize(cropped_reporter_image)
    cropped_nuclear_image = z_normalize(cropped_nuclear_image)
    
    # Calculate cross-correlation between cell-cycle reporter and nuclear channel images 
    cross_corr_frame = cross_correlation(cropped_reporter_image, cropped_nuclear_image)
    # Get Nuclei that are within the boundary of cell cytoplasm 
    nuclei_within_cell_mask = nuclei_within_cell_boundary(cropped_nuclear_mask, cropped_reporter_image,
                                                          cell_mask, hole_pixels = 3)
    if cross_corr_frame != 0.0:
       cross_corr.append(cross_corr_frame)
       frames_traj.append(fid)
    
    # Apply the nuclei mask to the reporter image
    nucleus_region = np.ma.masked_array(cropped_reporter_image, mask=~nuclei_within_cell_mask)
    cyto_region = np.ma.masked_array(cropped_reporter_image, mask=np.logical_and(nuclei_within_cell_mask, cell_mask))
    
    # Visualize the masked regions for debugging
    axs[1][0].imshow(cell_mask, cmap = plt.cm.gray)
    axs[1][0].imshow(cyto_region, cmap = plt.cm.viridis, alpha = 0.7)
    axs[1][0].imshow(nucleus_region, cmap = plt.cm.jet, alpha = 0.7)
    axs[1][0].axis('off')
    
    axs[1][1].plot(frames_traj, cross_corr, color = 'b', marker = 'o', markerfacecolor = 'w', markersize = 8)
    axs[1][1].set_xlabel('Frames')
    axs[1][1].set_ylabel('Cross-Correlation b/w reporter & nuclear channels')
     
    # After updating the content, reapply the tight layout
    fig.tight_layout()
    return axs

Save an Animation of a Single-Cell Trajectory: Cross-correlations, Phase image, Overlaid Reporter & Nuclear Images, and Embedding 

In [70]:
trajl = 2
border_margin = 2 # Pixel margins of cell-borders 
wctm.trajl = trajl
#tid = 8 # Trajectory number from the end of the array 
#full_traj_indc = indtrajs[-tid] # Full Trajectory Index 
full_traj_indc = 234
n_bins = 50
wctm.traj = modelSet[model_indc].get_traj_segments(trajl) # Get trajectory segments, to be consistent with trajectory embeddings
cell_traj = modelSet[model_indc].trajectories[full_traj_indc]
modelSet[model_indc].get_trajectory_embedding(trajl, embedding_type = 'UMAP', neigen = 2, get_trajectories = False)
x0 = modelSet[model_indc].Xtraj # Calculated from "get_trajectory_embedding()"
prob, xedges1, yedges1 = np.histogram2d(x0[:, 0], x0[:, 1], bins = n_bins, density = True) # Coordinate landscape probability distribution
xx, yy = np.meshgrid(.5*xedges1[1:] + .5*xedges1[0:-1], .5*yedges1[1:] + .5*yedges1[0:-1]) # Grid coordinates
# Get coordinates and cell indices along the trajectory set
xt, inds_traj = modelSet[model_indc].get_Xtraj_celltrajectory(cell_traj, Xtraj = x0, traj = wctm.traj) 
cell_traj = modelSet[model_indc].traj[inds_traj, -1]
n_trajs = cell_traj.size # Number of single-cell trajectory snippets

Plot of Cell Coordinates And Images Along Single Cell Trajectories

In [71]:
fig, axs = plt.subplots(2, 2, figsize = (12, 10))
fig.tight_layout()  # Adjust subplots to fit into the figure area
frames_traj = []
cross_corr = []
##################### CALL TO SAVE IMAGES AS AN ANIMATION ######################
try:
    anim = FuncAnimation(fig, update, frames = n_trajs, interval = 330)
    save_file = f"ccTrjEmbedding_{tmSet[0]}_{full_traj_indc}_trajl{trajl-1}.gif"
    #save_file = 'single_cell_trajectory_EGF'+str(save_file_indc)+'.mp4'
    anim.save(save_file, writer = 'pillow')
    #anim.save(save_file, writer = writer, dpi = 300)
    print(f"Animation saved as {save_file}")
except Exception as e:
    print(f"Error occurred: {e}")
plt.close(fig)

Cell indices: 50381
extracting cellborders from frame 191 image 0
extracting cellborders from frame 191 image 0
Animation saved as ccTrjEmbedding_EGF1_234_trajl1.gif
